In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
import logging
from sys import getsizeof

In [ ]:
#onset
def onset(site, year):                
    datafolder = '/home/hoyinchan/blue/Data/data2021/data2021/'
    home_directory = "/home/hoyinchan/code/AKI_CDM_PY/"
    pred_end = 7    

    # load tables    
#   onset = pd.read_csv(datafolder+site+'/raw/'+'AKI_ONSETS'+'.csv',sep=',', dtype=({'PATID': 'object', 'ENCOUNTERID': 'object'}))
    onset = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_onset_'+site+'.pkl')        
#    onset = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_onsetpy_'+site+'.pkl')        

#    years = list(pd.to_datetime(onset['ADMIT_DATE']).dt.year.unique())

    print('Running onset on site '+site+":"+str(year), flush = True)            
    #get paitient by year
    onset.loc[:,'ADMIT_DATE'] = pd.to_datetime(onset['ADMIT_DATE'])
    onset_yr = onset.query("ADMIT_DATE >= '"+str(year)+"/01/01' and ADMIT_DATE <= '"+str(year)+"/12/31'")
#    onset_yr = onset    #Still need to seperate by year if file too big

    # get non-AKI paitients
    onset_yr_aki0 = onset_yr[onset_yr["NONAKI_SINCE_ADMIT"].notnull()]
    onset_yr_aki0_select = onset_yr_aki0[["PATID", "ENCOUNTERID", "NONAKI_SINCE_ADMIT"]]
    onset_yr_aki0_select = onset_yr_aki0_select.assign(FLAG = 0)
    onset_yr_aki0_select = onset_yr_aki0_select >> rename(SINCE_ADMIT=X.NONAKI_SINCE_ADMIT)

    # Get AKI1 paitients    
    onset_yr_aki1 = onset_yr[np.logical_and(onset_yr["AKI1_SINCE_ADMIT"].notnull(), np.logical_and(onset_yr["AKI2_SINCE_ADMIT"].isnull(), onset_yr["AKI3_SINCE_ADMIT"].isnull()))]
    onset_yr_aki1_select = onset_yr_aki1[["PATID", "ENCOUNTERID", "AKI1_SINCE_ADMIT"]]
    onset_yr_aki1_select = onset_yr_aki1_select.assign(FLAG = 1)
    onset_yr_aki1_select = onset_yr_aki1_select >> rename(SINCE_ADMIT=X.AKI1_SINCE_ADMIT)
    
    # Get AKI2 paitients    
    onset_yr_aki2 = onset_yr[np.logical_and(onset_yr["AKI2_SINCE_ADMIT"].notnull(), onset_yr["AKI3_SINCE_ADMIT"].isnull())]
    onset_yr_aki2_select = onset_yr_aki2[["PATID", "ENCOUNTERID", "AKI2_SINCE_ADMIT"]]
    onset_yr_aki2_select = onset_yr_aki2_select.assign(FLAG = 2)
    onset_yr_aki2_select = onset_yr_aki2_select >> rename(SINCE_ADMIT=X.AKI2_SINCE_ADMIT)    

    # Get AKI3 paitients    
    onset_yr_aki3 = onset_yr[onset_yr["AKI3_SINCE_ADMIT"].notnull()]
    onset_yr_aki3_select = onset_yr_aki3[["PATID", "ENCOUNTERID", "AKI3_SINCE_ADMIT"]]
    onset_yr_aki3_select = onset_yr_aki3_select.assign(FLAG = 3)
    onset_yr_aki3_select = onset_yr_aki3_select >> rename(SINCE_ADMIT=X.AKI3_SINCE_ADMIT)        
    
    newdf = pd.concat([onset_yr_aki1_select, onset_yr_aki0_select, onset_yr_aki2_select, onset_yr_aki3_select], axis=0, sort=False).reset_index(drop=True)
#    newdf = pd.concat([onset_yr_aki1_select, onset_yr_aki0_select], axis=0, sort=False).reset_index(drop=True)
#    newdf = pd.concat([onset_yr_aki2_select, onset_yr_aki1_select], axis=0, sort=False).reset_index(drop=True)

    
    # This Table has to be straightly nonnull, (sometime the SINCE_ADMIT is null)
    # onset_yr_aki0_select = onset_yr_aki0_select.dropna()
    # onset_yr_aki1_select = onset_yr_aki1_select.dropna()
    # onset_yr_aki2_select = onset_yr_aki2_select.dropna()
    # onset_yr_aki3_select = onset_yr_aki3_select.dropna()
    
    newdf = newdf.dropna()
    #Save table   
    newdf.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')

    print('Finished onset on site '+site+":"+str(year), flush = True)

   # no need, just use that day
#   onset_yr_aki1_select.loc[onset_yr_aki1_select.AKI1_SINCE_ADMIT == 0, 'AKI1_SINCE_ADMIT'] = 1    
    
    #PATID = onset_yr_aki1_select['PATID'].tolist()
    #ENCOUNTERID = onset_yr_aki1_select['ENCOUNTERID'].tolist()
    #SINCE_ADMIT = onset_yr_aki1_select['AKI1_SINCE_ADMIT'].tolist()

    #Expand data (Create row for each day for each encounter until pred end)
    #newdf = pd.DataFrame(np.repeat(onset_yr_aki1_select.values,7,axis=0))
    #newdf.columns = onset_yr_aki1_select.columns
    #since_admit = [*range(7)]*onset_yr_aki1_select.shape[0]
    #newdf['SINCE_ADMIT'] = since_admit
    #conditions = [(newdf['SINCE_ADMIT'] < newdf['AKI1_SINCE_ADMIT']-1), (newdf['SINCE_ADMIT'] == newdf['AKI1_SINCE_ADMIT']-1), (newdf['SINCE_ADMIT'] > newdf['AKI1_SINCE_ADMIT']-1)]
    #values = [0, 1, -1]
    #newdf['FLAG'] = np.select(conditions, values)
    #newdf = newdf.drop(newdf[newdf['FLAG']==-1].index)

In [ ]:
if __name__ == "__main__":
    #onset
    sites = ['IUR', 'MCRI', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'UTSW']
    #site = ['UTSW', 'MCW', 'UofU']

    for site in sites:
        onset(site)

In [ ]:
# vital
def vital(site, year):
    print('Running vital on site '+site+":"+str(year), flush = True)

    # load tables
#    vital = pd.read_csv(datafolder+site+'/raw/'+'AKI_VITAL'+'.csv',sep=',', dtype=({'PATID': 'object', 'ENCOUNTERID': 'object'}))
#    vital = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_vital_'+site+'.pkl')
    vital = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_vital_'+site+'.pkl')
    
    #use only relevent data
#    newdfX = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
    newdfX = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
#    newdfX = newdfX >> select('PATID', 'ENCOUNTERID') >> mutate(dummy = True) >> distinct()
#    vital = (pd.merge(vital, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy)).reset_index(drop=True)
    newdfX = newdfX >> select('PATID', 'ENCOUNTERID', 'SINCE_ADMIT') >> mutate(dummy = True)
    vital = (pd.merge(vital, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy) >> mutate(FUTURE=X.SINCE_ADMIT-X.DAYS_SINCE_ADMIT)).reset_index(drop=True)

    #0 hours prediction
#    vital = vital[vital['FUTURE']>=0].drop(['SINCE_ADMIT','FUTURE'],axis=1)    
    
    #24 hours prediction
    vital = vital[vital['FUTURE']>0].drop(['SINCE_ADMIT','FUTURE'],axis=1)
        
    #Rethink if I need it TODO (no need since fill last data)
    #vital.loc[vital.DAYS_SINCE_ADMIT == 0, 'DAYS_SINCE_ADMIT'] = 1

    # drop unused column
#    vital = vital.drop(['Row #','MEASURE_DATE_TIME','SMOKING', 'TOBACCO', 'TOBACCO_TYPE', 'WT'],axis=1, errors='ignore')

    #Calculate daily average
    vital_mean = vital.groupby(['PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT']).mean().reset_index()

    #Transform vital Table (Row over the previous value if unknown) (Continuous)
    vital_list = []
    #depreciated vital table only collect data until onset
    #vital_p = (vital_mean >> mask(X.DAYS_SINCE_ADMIT < 0) >> drop('DAYS_SINCE_ADMIT') >> distinct())
    vital_sys = vital_mean >> select('PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT', 'SYSTOLIC')
    vital_dia = vital_mean >> select('PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT', 'DIASTOLIC')
    vital_bmi = vital_mean >> select('PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT', 'ORIGINAL_BMI')
    vital_wt = vital_mean >> select('PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT', 'WT')

    # deprecitaed only collect at onset
#    for i in range(pred_end+1):
#        vital_sys_p = vital_sys.loc[((vital_sys >> mask(X.DAYS_SINCE_ADMIT <= i)).dropna()).groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)
#        vital_dia_p = vital_dia.loc[((vital_dia >> mask(X.DAYS_SINCE_ADMIT <= i)).dropna()).groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)
#        vital_bmi_p = vital_bmi.loc[((vital_bmi >> mask(X.DAYS_SINCE_ADMIT <= i)).dropna()).groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)
#        vital_wt_p = vital_wt.loc[((vital_wt >> mask(X.DAYS_SINCE_ADMIT <= i)).dropna()).groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)        
        # fill in previous day value (row over if unknown)
#        vital_p = pd.merge(vital_sys_p, vital_dia_p, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='outer')
#        vital_p = pd.merge(vital_p, vital_bmi_p, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='outer')
#        vital_p = pd.merge(vital_p, vital_wt_p, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='outer')        
#        vital_p = vital_p >> mutate(SINCE_ADMIT = i)
#        vital_list.append(vital_p.copy())
#    vital_t = pd.concat(vital_list, axis=0, ignore_index=True) 
    #Save table
#    vital_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/vital_'+site+'_'+str(year)+'.pkl')

    # idxmax is slow, use sort and last
    # vital_sys_p = vital_sys.loc[vital_sys.dropna().groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)
    # vital_dia_p = vital_dia.loc[vital_dia.dropna().groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)
    # vital_bmi_p = vital_bmi.loc[vital_bmi.dropna().groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)
    # vital_wt_p  =  vital_wt.loc[ vital_wt.dropna().groupby(['PATID', 'ENCOUNTERID']).DAYS_SINCE_ADMIT.idxmax()] >> select(~X.DAYS_SINCE_ADMIT)        

    vital_sys_p = vital_sys.dropna().sort_values(['PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT']).groupby(['PATID', 'ENCOUNTERID']).agg({'SYSTOLIC':'last'}).reset_index()    
    vital_dia_p = vital_dia.dropna().sort_values(['PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT']).groupby(['PATID', 'ENCOUNTERID']).agg({'DIASTOLIC':'last'}).reset_index()    
    vital_bmi_p = vital_bmi.dropna().sort_values(['PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT']).groupby(['PATID', 'ENCOUNTERID']).agg({'ORIGINAL_BMI':'last'}).reset_index()    
    vital_wt_p  =  vital_wt.dropna().sort_values(['PATID', 'ENCOUNTERID', 'DAYS_SINCE_ADMIT']).groupby(['PATID', 'ENCOUNTERID']).agg({'WT':'last'}).reset_index()    

    
# fill in previous day value (row over if unknown)
    vital_t = pd.merge(vital_sys_p, vital_dia_p, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='outer')
    vital_t = pd.merge(vital_t, vital_bmi_p, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='outer')
    vital_t = pd.merge(vital_t, vital_wt_p, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='outer')        

    #Save table
#    vital_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/vital_'+site+'_'+str(year)+'.pkl')    
    vital_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/vital_'+site+'_'+str(year)+'.pkl')

    #consistency check
    if vital_t.empty:
        logging.basicConfig(filename='vital.log', filemode='a')    
        print('DATAFRAME EMPTY!!!!!! '+site+":"+str(year), flush = True)
        logging.error('vital: DATAFRAME EMPTY!!!!!! '+site+":"+str(year))
        logging.shutdown()

    print('Finished vital on site '+site+":"+str(year), flush = True)
#    return vital_t

In [ ]:
def demo(site, year):
    # demo
    print('Running demo on site '+site+":"+str(year), flush = True)

    # load tables
#    demo = pd.read_csv(datafolder+site+'/raw/'+'AKI_DEMO'+'.csv',sep=',', dtype=({'PATID': 'object', 'ENCOUNTERID': 'object'}))
    demo = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_demo_'+site+'.pkl')
    
    #use only relevent data
    newdfX = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
    newdfX = newdfX >> select('PATID', 'ENCOUNTERID') >> mutate(dummy = True) >> distinct()
    demo = (pd.merge(demo, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy)).reset_index(drop=True)

    # drop unused column
#    demo = demo.drop(['Row #','DEATH_DATE','BIRTH_DATE','DDAYS_SINCE_ENC','DEATH_DATE_IMPUTE','DEATH_SOURCE'],axis=1, errors='ignore')

    #onehot transform demo 
    var = ['SEX', 'RACE', 'HISPANIC']
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(demo[var])
    demo_onehot_cat = pd.DataFrame(enc.transform(demo[var]).toarray(), columns=enc.get_feature_names(var)).astype('bool')
    demo_one = pd.concat([demo[['PATID', 'ENCOUNTERID', 'AGE']].reset_index(), demo_onehot_cat], axis=1).drop('index',axis=1)    
    
    #Save table
    demo_one.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/demo_'+site+'_'+str(year)+'.pkl')

    #consistency check
    if demo_one.empty:
        logging.basicConfig(filename='demo.log', filemode='a')    
        print('DATAFRAME EMPTY!!!!!! '+site+":"+str(year), flush = True)
        logging.error('demo: DATAFRAME EMPTY!!!!!! '+site+":"+str(year))
        logging.shutdown()

    print('Finished demo on site '+site+":"+str(year), flush = True)

In [ ]:
def dx(site, year):
    home_directory = "/home/hoyinchan/code/AKI_CDM_PY/"
    
    # dx
    print('Running dx on site '+site+":"+str(year), flush = True)

    # load table
#    dx = pd.read_csv(datafolder+site+'/raw/'+'AKI_DX'+'.csv',sep=',', dtype=({'PATID': 'object', 'ENCOUNTERID': 'object', 'DX_TYPE': 'object', 'DX': 'object'}))
    dx = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_dx_'+site+'.pkl')
    
    #use only relevent data
    newdfX = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
    #newdfX = newdfX >> select('PATID', 'ENCOUNTERID') >> mutate(dummy = True) >> distinct()
    #dx = (pd.merge(dx, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy)).reset_index(drop=True)
    newdfX = newdfX >> select('PATID', 'ENCOUNTERID', 'SINCE_ADMIT') >> mutate(dummy = True)
    dx = (pd.merge(dx, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy) >> mutate(FUTURE=X.SINCE_ADMIT-X.DAYS_SINCE_ADMIT)).reset_index(drop=True)

    #0 hours prediction
#    dx = dx[dx['FUTURE']>=0].drop(['SINCE_ADMIT','FUTURE'],axis=1)

    #24 hours prediction
    dx = dx[dx['FUTURE']>0].drop(['SINCE_ADMIT','FUTURE'],axis=1)
    
    
    #Some site use 9 some site use 09
    dx['DX_TYPE'] = dx['DX_TYPE'].where(dx['DX_TYPE'] != '9', '09')
    
    # ICD10 -> ICD09
    icd10toicd09 = pd.read_csv(home_directory+'2018_I10gem.csv',sep=',')
    
    icd10toicd09.columns = ['DX', 'DX09']
    #add fillna for unmatch ICD10 (DONE)
    #dx3 = dx >> mask(X.DX_TYPE == '10') >> left_join(icd10toicd09, by='DX') >> mutate(DX_TYPE = '09', DX = X.DX09) >> select(~X.DX09)
#    dx4 = dx >> mask(X.DX_TYPE == '10') >> left_join(icd10toicd09, by='DX')
    dx4 = dx >> mask(X.DX_TYPE == '10')
    dx4['DX'] = dx4['DX'].map(lambda x: x.replace('.',''))
    dx4 = dx4 >> left_join(icd10toicd09, by='DX')
    #Keep icd10 if no match
    dx4['DX_TYPE'] = dx4['DX_TYPE'].where(dx4['DX09'].isnull(), '09')
    dx4['DX'] = dx4['DX'].where(dx4['DX09'].isnull(), dx4['DX09'])
    dx4 = dx4.drop('DX09', axis=1)
    dx = pd.concat([dx >> mask(X.DX_TYPE != '10'), dx4], axis=0)

    # Roll icd 09 code up native
    dx['DX'] = dx['DX'].where(dx['DX_TYPE'] != '09', dx['DX'].map(lambda x: x[0:3]))

    # Roll icd 10 code up native
    dx['DX'] = dx['DX'].where(dx['DX_TYPE'] != '10', dx['DX'].map(lambda x: x[0:3]))
    
    #Rethink if I need it TODO (no need since historic)
    #dx.loc[dx.DAYS_SINCE_ADMIT == 0, 'DAYS_SINCE_ADMIT'] = 1

    # drop unused column
#    dx = dx.drop(['DX_SOURCE','DX_ORIGIN','PDX','DX_DATE'],axis=1, errors='ignore')

    #Transform dx table (Historical data: Yes if any diagnoasis show up) (Assuming all DAYS_SINCE_ADMIT < 0) (Boolean)
    
    dx['sixmonth'] = '<6'
    dx['sixmonth'] = dx['sixmonth'].where(dx['DAYS_SINCE_ADMIT']<-365/2, '>6') # becareful negative number
    dx_t = dx >> mutate(DX='DX:'+X.DX_TYPE+":"+X.DX+X.sixmonth) >> drop('DX_TYPE') >> drop('sixmonth')
    dx_t = (dx_t >> drop('DAYS_SINCE_ADMIT') >> mutate(dummy = True) >> distinct()).pivot(index=['PATID', 'ENCOUNTERID'], columns='DX', values='dummy').fillna(False).reset_index()

    #Save table
    dx_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/dx_'+site+'_'+str(year)+'.pkl')

    #consistency check
    if dx_t.empty:
        logging.basicConfig(filename='dx.log', filemode='a')
        print('DATAFRAME EMPTY!!!!!! '+site+":"+str(year), flush = True)
        logging.error('dx: DATAFRAME EMPTY!!!!!! '+site+":"+str(year))
        logging.shutdown()
    
    #consistency check2
        
    print('Finished dx on site '+site+":"+str(year), flush = True)

In [ ]:
def px(site, year):
    # px
    print('Running px on site '+site+":"+str(year), flush = True)

    #load table
#    px = pd.read_csv(datafolder+site+'/raw/'+'AKI_PX'+'.csv',sep=',', dtype=({'PATID': 'object', 'ENCOUNTERID': 'object'}))
    px = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_px_'+site+'.pkl')
    
    #use only relevent data
    newdfX = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
    #newdfX = newdfX >> select('PATID', 'ENCOUNTERID') >> mutate(dummy = True) >> distinct()
    #px = (pd.merge(px, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy)).reset_index(drop=True)
    newdfX = newdfX >> select('PATID', 'ENCOUNTERID', 'SINCE_ADMIT') >> mutate(dummy = True)
    px = (pd.merge(px, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy) >> mutate(FUTURE=X.SINCE_ADMIT-X.DAYS_SINCE_ADMIT)).reset_index(drop=True)

    #0 hours prediction
#    px = px[px['FUTURE']>=0].drop(['SINCE_ADMIT','FUTURE'],axis=1)

    #24 hours prediction
    px = px[px['FUTURE']>0].drop(['SINCE_ADMIT','FUTURE'],axis=1)
    
    
    #Some site use 9 some site use 09
    px['PX_TYPE'] = px['PX_TYPE'].where(px['PX_TYPE'] != '9', '09')
    
    #Rethink if I need it TODO (exact date, so follow onset) (actually no, because day 0 is still day0, onset just shift theoneset to next day)
#    px.loc[px.DAYS_SINCE_ADMIT == 0, 'DAYS_SINCE_ADMIT'] = 1

    # drop unused column
    px = px >> mutate(PX='PX:'+X.PX_TYPE+":"+X.PX) >> drop('PX_TYPE')
#    px = px.drop(['Row #', 'PX_SOURCE','DX_ORIGIN','PDX','PX_DATE', 'PX_TYPE'],axis=1, errors='ignore')

    #Depreciate -> Chnage back to historic
    #Transform PX Table (Exact Date) (Boolean)
#    px_list = []
#    for i in range(pred_end+1):
#        px_p = (px >> mask(X.DAYS_SINCE_ADMIT == i) >> drop('DAYS_SINCE_ADMIT') >> mutate(dummy = True) >> distinct()).pivot(index=['PATID', 'ENCOUNTERID'], columns='PX', values='dummy').fillna(False).reset_index()
#        px_p = px_p >> mutate(SINCE_ADMIT = i)
#        px_list.append(px_p.copy())
#    px_t = pd.concat(px_list, axis=0, ignore_index=True).fillna(False) 

    px_t = (px >> drop('DAYS_SINCE_ADMIT') >> mutate(dummy = True) >> distinct()).pivot(index=['PATID', 'ENCOUNTERID'], columns='PX', values='dummy').fillna(False).reset_index()

    #Save table
    px_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/px_'+site+'_'+str(year)+'.pkl')

    if px_t.empty:
        logging.basicConfig(filename='px.log', filemode='a')
        print('DATAFRAME EMPTY!!!!!! '+site+":"+str(year), flush = True)
        logging.error('px: DATAFRAME EMPTY!!!!!! '+site+":"+str(year))
        logging.shutdown()


    print('Finished px on site '+site+":"+str(year), flush = True)

In [ ]:
def lab(site, year):
#site = 'UPITT'
#year=2018
#if True:
    # lab
    print('Running lab on site '+site+":"+str(year), flush = True)

    #load table
#    lab = pd.read_csv(datafolder+site+'/raw/'+'AKI_LAB'+'.csv',sep=',', dtype=({'PATID': 'object', 'ENCOUNTERID': 'object'}))
#    lab = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_lab_'+site+'.pkl')
    lab = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_lab_'+site+'.pkl')
    #Rethink if I need it TODO ( no need since row over)
    #lab.loc[lab.DAYS_SINCE_ADMIT == 0, 'DAYS_SINCE_ADMIT'] = 1
   
    #use only relevent data
    newdfX = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
    #newdfX = newdfX >> select('PATID', 'ENCOUNTERID') >> mutate(dummy = True) >> distinct()
    #lab = (pd.merge(lab, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy)).reset_index(drop=True)
    newdfX = newdfX >> select('PATID', 'ENCOUNTERID', 'SINCE_ADMIT') >> mutate(dummy = True)
    lab = (pd.merge(lab, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy) >> mutate(FUTURE=X.SINCE_ADMIT-X.DAYS_SINCE_ADMIT)).reset_index(drop=True)

    #0 hours prediciton
#    lab = lab[lab['FUTURE']>=0].drop(['SINCE_ADMIT','FUTURE'],axis=1)

    #24 hours prediciton
    lab = lab[lab['FUTURE']>0].drop(['SINCE_ADMIT','FUTURE'],axis=1)


    #seperate into numberic lab and pos/neg lab
    #calculate categorical first
    lab_cat = lab.loc[lab['RESULT_NUM'].isnull()]
    if lab_cat.empty:
        labcat_t = lab >> select('PATID','ENCOUNTERID')
        labcat_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/labcat_'+site+'_'+str(year)+'.pkl')
    else:
        #Calculate daily average    
#        lab_mode = lab_cat.loc[:, ['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'DAYS_SINCE_ADMIT', 'RESULT_QUAL']].groupby(['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'DAYS_SINCE_ADMIT']).agg(mymode).reset_index()
        
        lab_mode = lab_cat.loc[:, ['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'DAYS_SINCE_ADMIT', 'RESULT_QUAL']].groupby(['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'DAYS_SINCE_ADMIT']).agg(pd.Series.mode).reset_index()
        lab_mode_nnd = lab_mode.loc[lab_mode['RESULT_QUAL'].apply(type) == str].copy()
        lab_mode_nd = lab_mode.loc[lab_mode['RESULT_QUAL'].apply(type) != str].copy()
        pattern = '[\[\]\']'
        lab_mode_nd.loc[:,'RESULT_QUAL'] = lab_mode_nd['RESULT_QUAL'].apply(lambda x: re.sub(pattern, "", np.array2string(x,separator='-')))
        lab_mode = pd.concat([lab_mode_nd, lab_mode_nnd], ignore_index=True)

        labcat_t = lab_mode.sort_values(['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'DAYS_SINCE_ADMIT']).groupby(['PATID', 'ENCOUNTERID', 'LAB_LOINC']).agg({'RESULT_QUAL':'last'}).reset_index()
        labcat_t = labcat_t >> mutate(LAB_LOINC='LAB:'+":"+X.LAB_LOINC+"("+X.RESULT_QUAL+")") >> mutate(dummy = True) >> select('PATID', 'ENCOUNTERID', 'LAB_LOINC', 'dummy')
        labcat_t = labcat_t.pivot(index=['PATID', 'ENCOUNTERID'], columns='LAB_LOINC', values='dummy').fillna(False).reset_index()        
        #Save table    
        labcat_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/labcat_'+site+'_'+str(year)+'.pkl') 
    
    #calculate numerica    
    lab_num = lab.loc[lab['RESULT_NUM'].notnull()]  
    if lab_num.empty:
        labnum_t = lab >> select('PATID','ENCOUNTERID')
#        labnum_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/labnum_'+site+'_'+str(year)+'.pkl')
        labnum_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/labnum_'+site+'_'+str(year)+'.pkl')        
    else:    
        #Calculate daily average
        lab_mean = lab_num.groupby(['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'RESULT_UNIT', 'DAYS_SINCE_ADMIT']).agg({'RESULT_NUM':'mean'}).reset_index()
        lab_mean = lab_mean >> mutate(LAB_LOINC='LAB:'+":"+X.LAB_LOINC+"("+X.RESULT_UNIT+")")
        labnum_t = lab_mean.sort_values(['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'DAYS_SINCE_ADMIT']).groupby(['PATID', 'ENCOUNTERID', 'LAB_LOINC']).agg({'RESULT_NUM':'last'}).reset_index()
        labnum_t = labnum_t.pivot(index=['PATID', 'ENCOUNTERID'], columns='LAB_LOINC', values='RESULT_NUM').reset_index()
        #Save table
#        labnum_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/labnum_'+site+'_'+str(year)+'.pkl')        
        labnum_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/labnum_'+site+'_'+str(year)+'.pkl')

    #Depreciate -> Chnage back to historic
    #Transform LAB Table (Row over the previous value if unknown) (Continuous)
#    lab_list = []
    # use 0 as beginning #lab.loc[lab.DAYS_SINCE_ADMIT == 0, 'DAYS_SINCE_ADMIT'] = 1 disabled
#    lab_p = (lab_mean >> mask(X.DAYS_SINCE_ADMIT == 0) >> drop('DAYS_SINCE_ADMIT') >> distinct()).pivot(index=['PATID', 'ENCOUNTERID'], columns='LAB_LOINC', values='RESULT_NUM').reset_index()
#    lab_p = lab_p >> mutate(SINCE_ADMIT = 1)   
#    lab_list.append(lab_p.copy())
    # use 1 as beginning #lab.loc[lab.DAYS_SINCE_ADMIT == 0, 'DAYS_SINCE_ADMIT'] = 1 disabled
#    for i in range(1,pred_end+1):
#        lab_p = (lab_mean >> mask(X.DAYS_SINCE_ADMIT == i) >> drop('DAYS_SINCE_ADMIT') >> distinct()).pivot(index=['PATID', 'ENCOUNTERID'], columns='LAB_LOINC', values='RESULT_NUM').reset_index()
        # fill in previous day value (row over if unknown)
#        lab_p = lab_p.join(lab_list[-1], lsuffix="DROP").filter(regex="^(?!.*DROP)")
#        lab_p = lab_p.combine_first(lab_list[-1])
#        lab_p = lab_p >> mutate(SINCE_ADMIT = i)    
#        lab_list.append(lab_p.copy())
#    lab_t = pd.concat(lab_list, axis=0, ignore_index=True) 

    if labnum_t.empty or labcat_t.empty:
        logging.basicConfig(filename='lab.log', filemode='a')
        print('DATAFRAME EMPTY!!!!!! '+site+":"+str(year), flush = True)
        logging.error('lab: DATAFRAME EMPTY!!!!!! '+site+":"+str(year))
        logging.shutdown()   

    print('Finished lab on site '+site+":"+str(year), flush = True)

In [ ]:
def amed(site, year):
    # amed
    home_directory = "/home/hoyinchan/code/AKI_CDM_PY/"
    print('Running amed on site '+site+":"+str(year), flush = True)

    #load table
#    amed = pd.read_csv(datafolder+site+'/raw/'+'AKI_AMED'+'.csv',sep=',', dtype=({'PATID': 'object', 'ENCOUNTERID': 'object'}))
    amed = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_amed_'+site+'.pkl')
    
    #use only relevent data
    newdfX = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/onset_'+site+'_'+str(year)+'.pkl')
    #newdfX = newdfX >> select('PATID', 'ENCOUNTERID') >> mutate(dummy = True) >> distinct()
    #amed = (pd.merge(amed, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy)).reset_index(drop=True)
    newdfX = newdfX >> select('PATID', 'ENCOUNTERID', 'SINCE_ADMIT') >> mutate(dummy = True)
    amed = (pd.merge(amed, newdfX, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left').fillna({'dummy': False}) >> mask(X.dummy) >> select(~X.dummy) >> mutate(FUTURE=X.SINCE_ADMIT-X.DAYS_SINCE_ADMIT)).reset_index(drop=True)

    #0 hours prediction
#    amed = amed[amed['FUTURE']>=0].drop(['SINCE_ADMIT','FUTURE'],axis=1)

    #24 hours prediction
    amed = amed[amed['FUTURE']>0].drop(['SINCE_ADMIT','FUTURE'],axis=1)

    # rxnorm -> atc
    amed_rx = amed.loc[amed['MEDADMIN_TYPE'] == "RX"]
    if not amed_rx.empty:
        # pd.DataFrame(amed['MEDADMIN_CODE'].unique()).to_csv('/home/hchan2/AKI/AKI_Python/data/'+site+'/rxnormtmp.csv', sep=',', index=False, header = False)
        # Go to run rxnorm2atcR.ipynb NOW
        rxcui2atc_dtypes =  {"Rxcui": 'object', "ATC4th": 'object'}    
        rxcui2atc = pd.read_csv('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/rxnorm_out_'+site+'.csv',sep=',', dtype=(rxcui2atc_dtypes)) >> rename(MEDADMIN_CODE=X.Rxcui)
        amed_rx = amed_rx >> left_join(rxcui2atc, by='MEDADMIN_CODE')
        amed_rx['MEDADMIN_TYPE'] = amed_rx['MEDADMIN_TYPE'].where(amed_rx['ATC4th'].isnull(), 'ATC')
        amed_rx['MEDADMIN_CODE'] = amed_rx['MEDADMIN_CODE'].where(amed_rx['ATC4th'].isnull(), amed_rx['ATC4th'])
        amed_rx = amed_rx >> mutate(MEDADMIN_CODE='MED:'+X.MEDADMIN_TYPE+':'+X.MEDADMIN_CODE)
        amed_rx = amed_rx >> select('PATID', 'ENCOUNTERID', 'MEDADMIN_CODE', 'DAYS_SINCE_ADMIT')
    else:
        amed_rx = amed_rx >> select('PATID', 'ENCOUNTERID', 'MEDADMIN_CODE', 'DAYS_SINCE_ADMIT')
    
    # ndc -> atc
    amed_ndc = amed.loc[amed['MEDADMIN_TYPE'] == "ND"]    
    if not amed_ndc.empty:    
        # pd.DataFrame(amed['MEDADMIN_CODE'].unique()).to_csv('/home/hchan2/AKI/AKI_Python/data/'+site+'/rxnormtmp.csv', sep=',', index=False, header = False)
        # Go to run rxnorm2atcR.ipynb NOW
        ndc2atc_dtypes =  {"ndc": 'object', "ATC4th": 'object'}    
        ndc2atc = pd.read_csv('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/ndc_out_'+site+'.csv',sep=',', dtype=(ndc2atc_dtypes)) >> rename(MEDADMIN_CODE=X.ndc)
        amed_ndc = amed_ndc >> left_join(ndc2atc, by='MEDADMIN_CODE')
        amed_ndc['MEDADMIN_TYPE'] = amed_ndc['MEDADMIN_TYPE'].where(amed_ndc['ATC4th'].isnull(), 'ATC')
        amed_ndc['MEDADMIN_CODE'] = amed_ndc['MEDADMIN_CODE'].where(amed_ndc['ATC4th'].isnull(), amed_ndc['ATC4th'])
        amed_ndc = amed_ndc >> mutate(MEDADMIN_CODE='MED:'+X.MEDADMIN_TYPE+':'+X.MEDADMIN_CODE)
        amed_ndc = amed_ndc >> select('PATID', 'ENCOUNTERID', 'MEDADMIN_CODE', 'DAYS_SINCE_ADMIT')
    else:
        amed_ndc = amed_ndc >> select('PATID', 'ENCOUNTERID', 'MEDADMIN_CODE', 'DAYS_SINCE_ADMIT')
       
    amed = pd.concat([amed_rx, amed_ndc], axis=0, ignore_index=True)   

    #Rethink if I need it TODO (yes since exact date) (actually no, because day 0 is still day0, onset just shift theoneset to next day)
#    amed.loc[amed.DAYS_SINCE_ADMIT == 0, 'DAYS_SINCE_ADMIT'] = 1

    #Depreciate -> Chnage back to historic    
    #Transform AMED Table (Exact date) (Boolean)
#    amed_list = []
#    for i in range(1,pred_end+1):
#        amed_p = (amed >> mask(X.DAYS_SINCE_ADMIT == i) >> drop('DAYS_SINCE_ADMIT') >> mutate(dummy = True) >> distinct()).fillna('NI').pivot(index=['PATID', 'ENCOUNTERID'], columns='ATC4th', values='dummy').fillna(False).reset_index()
#        amed_p = amed_p >> mutate(SINCE_ADMIT = i)
#        amed_list.append(amed_p.copy())
#    amed_t = pd.concat(amed_list, axis=0, ignore_index=True).fillna(False) 
    #TODO test if not amed data in onset paitient
    # [x for x in newdf['PATID'] if x in amed_t['PATID']]  

    amed_t = (amed >> drop('DAYS_SINCE_ADMIT') >> mutate(dummy = True) >> distinct()).pivot(index=['PATID', 'ENCOUNTERID'], columns='MEDADMIN_CODE', values='dummy').fillna(False).reset_index()
        
    #Save table
    amed_t.to_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/amed_'+site+'_'+str(year)+'.pkl')

    if amed_t.empty:
        logging.basicConfig(filename='amed.log', filemode='a')
        print('DATAFRAME EMPTY!!!!!! '+site+":"+str(year), flush = True)
        logging.error('amed: DATAFRAME EMPTY!!!!!! '+site+":"+str(year))
        logging.shutdown()   

    print('Finished amed on site '+site+":"+str(year), flush = True)